# simulation with ivs data

### Imports
Import the required libraries

In [2]:
# package(s) related to time, space and id
import datetime, time
import platform
import random
import os
import pathlib

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import pyproj
import shapely.geometry
from shapely.geometry import Point
import shapely
import geopandas as gpd
import movingpandas as mpd

# package(s) for data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# OpenTNSIM
import opentnsim
import opentnsim.core as core
import opentnsim.graph_module as graph_module
import opentnsim.plot as plot


# dtv_backend
import dtv_backend.fis as fis
import dtv_backend.network
import dtv_backend.network.network_utilities
import dtv_backend.postprocessing
import dtv_backend.simple
import dtv_backend.simulate

# Used for mathematical functions
import math             
import numpy as np

# Used for making the graph to visualize our problem
import networkx as nx  

### Create graph

load graph

In [3]:
url = "https://zenodo.org/record/6673604/files/network_digital_twin_v0.3.pickle?download=1"
graph1 = dtv_backend.fis.load_fis_network(url)


URLError: <urlopen error [Errno 11001] getaddrinfo failed>

remove edges without classification

In [ ]:
#remove edges without classification
for edge in graph1.edges:
    if str(graph1.edges[edge]['Classification']) == 'nan':
        graph1.remove_edge(edge[0], edge[1])

save graph object 

In [ ]:
df_edges = nx.to_pandas_edgelist(graph1)
gpd_edges = gpd.GeoDataFrame(df_edges)
gpd_edges.to_file("plots_routes/only_edges_with_classification.shp")

D:\Users\ALPF\AppData\Local\Temp\ipykernel_23120\1138430104.py:3: UserWarning:

Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.



In [7]:
[edge.classification for edge in graph1.edges]

NameError: name 'graph1' is not defined

### Make vessels and with route

#### Create vessels

In [ ]:
def shortest_path_restricted(graph, start_node, end_node, scheepstype_rws):
    """This function finds the shortest path for a ship from the start to the end node. 
    It is restricted to the edges with classifications fitting the scheepstype."""
    	
    okay_classifications = [classification]
    for edge in graph1.edges:
    if str(graph1.edges[edge]['Classification']) == 'nan':
        graph1.remove_edge(edge[0], edge[1])


In [6]:
# Make a class out of mix-ins
TransportResource = type('TransportResource', 
                         (core.Identifiable, core.ContainerDependentMovable, 
                          core.HasResource, core.Routable,
                          core.VesselProperties,
                         core.ExtraMetadata), 
                         {})

#define speed: 
def compute_v_provider(v_empty, v_full):
    return lambda x: 1

vessels = []
for ship_type in ["M6", "M10"]:
    vessel = TransportResource(**{
    "env" : None,
    "name": ship_type, 
    "type": ship_type,
    "B": 1, 
    "L": 10,
    "route": None,
    "geometry": None,  # lon, lat
    "capacity": 10,
    "v": 1,
    "compute_v": compute_v_provider(v_empty=1, v_full=1),
    "begin_node": graph1.nodes.get("8861095"),
    "eind_node": graph1.nodes.get("8861687")
    })
    vessel.geometry =vessel.begin_node.geometry
    vessel.route = get_route_v1(ship_type, start, end)


NameError: name 'graph1' is not defined

### Define general outline of env and simulation

In [282]:
def start(env, vessel):
    while True:
        vessel.log_entry_v0("Start sailing", env.now, "", vessel.geometry)
        yield from vessel.move()
        vessel.log_entry_v0("Stop sailing", env.now, "", vessel.geometry)
        
        if vessel.geometry == nx.get_node_attributes(env.FG, "geometry")[vessel.route[-1]]:
            break

In [283]:
# Start simpy environment
simulation_start = datetime.datetime.now()
env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))
env.epoch = time.mktime(simulation_start.timetuple())


# Add graph to environment
graph.add_resources(list(graph.graph.edges), np.ones(len(list(graph.graph.edges))), env)



In [284]:
vessels = vessels
for i, vessel in enumerate(vessels):
    # Add environment and path to the vessel
    vessel.env = env

    # Add the movements of the vessel to the simulation
    env.process(start(env, vessel))




### Run code for original graph

In [285]:
env.FG = graph.graph.copy()
env.epoch = datetime.datetime.now()

env.run()

##### Obtain vessel log information
The cel below uses the vessel log. The core function *log_entry* is used, which takes four arguments:

- **Log.** A text to describe what is logged.
- **t.** The timestamp.
- **Value.**  The value for the log (for sailing this is the distance).
- **Geometry** The location of the vessel while loggin.

In [286]:
vessel_log = gpd.GeoDataFrame(vessels[0].logbook, geometry='Geometry')
vessel_log = mpd.Trajectory(vessel_log, traj_id='vessel_1_trip_1', obj_id='vessel_1', t='Timestamp')
#vessel_log.to_line_gdf()
vessel_log.to_line_gdf().to_file('plots_routes/original_graph_logboek_movingpandas.gpkg')

d:\Users\ALPF\AppData\Local\pypoetry\Cache\virtualenvs\dtv-backend-pvs5eNot-py3.10\lib\site-packages\movingpandas\trajectory.py:166: MissingCRSWarning:

Trajectory generated without CRS. Computations will use Euclidean distances.



### Run code for adapted graph

### Visualization of path
If you get an error regarding ffmpeg use [this](https://stackoverflow.com/questions/13316397/matplotlib-animation-no-moviewriters-available) answer. You have to install ffmpeg in your Conda environment. It can be done using the following command.

```bash
#Install ffmpeg using Conda
conda install -c conda-forge ffmpeg
```

In [250]:
# # visualise vessel movements based on the information included in the vessel.log
# plot.vessel_kml(env, vessels, stepsize = 60)
# plot.graph_kml(env)
